In [1717]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from functools import reduce


"""
df
df = df_veg.merge(df_anim, left_on='Pays', right_on='Pays')

"""




names = ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément',
         'Élément', 'Code Produit', 'Produit', 'Code Année', 'Année', 'Unité',
         'Valeur', 'Symbole', 'Description du Symbole']


fn_population = 'fao_2013/FAOSTAT_2013_population.csv'
fn_animal = 'fao_2013/FAOSTAT_2013_animal.csv'
fn_vegetal = 'fao_2013/FAOSTAT_2013_vegetal.csv'

animaux = pd.read_csv(fn_animal, names=names, header=0)
vegetaux = pd.read_csv(fn_vegetal, names=names, header=0)
df_pop = pd.read_csv(fn_population, names=names, header=0)
df_pop = df_pop[df_pop.Pays != 'China']
print(df_pop.shape)


# Ajout d'une colonne 'Origine' dans animaux et vegetaux
animaux['Origine'] = 'Animale'
vegetaux['Origine'] = 'Végétale'

# Union de Végétaux et Animaux
aliments = pd.concat([vegetaux, animaux])

# drop 

# Projection
aliments = aliments[[
    'Pays',
    'Code Pays',
    'Produit',
    'Code Produit',
    'Origine',
    'Élément',
    'Valeur',
    'Année'
]]

aliments = aliments[aliments.Pays != 'Chine']
# Table pivot sur aliments
aliments = aliments.pivot_table('Valeur', index=['Code Pays','Pays','Produit','Code Produit','Année','Origine'], columns='Élément')

# Reset index
aliments.reset_index(inplace=True)

# Joindre population et aliments
aliments = pd.merge(aliments, df_pop[['Pays', 'Valeur']], left_on='Pays', right_on='Pays')

# Renommer les colonnes
aliments.rename(columns={
    'Aliments pour animaux' : 'alim_ani',
    'Autres utilisations (non alimentaire)' : 'autres_utilisations',
    'Semences' : 'semences',
    'Pertes' : 'pertes',
    'Nourriture' : 'nourriture',
    'Traitement' : 'transfo', # ?
    'Disponibilité intérieure' : 'dispo_int',
    'Disponibilité alimentaire (Kcal/personne/jour)' : 'dispo_alim_kcal_p_j',
    'Disponibilité de protéines en quantité (g/personne/jour)' : 'dispo_prot',
    'Disponibilité de matière grasse en quantité (g/personne/jour)' : 'dispo_mat_gr',
    'Valeur' : 'population'
}, inplace=True)
aliments.to_csv('fao_2013/aliments.csv')
# Remplacer les Nan
aliments.replace(np.nan, 0, inplace=True)

# Affichage
print('Résultat : ', aliments.shape[1], 'colonnes, ', aliments.shape[0], 'lignes')
aliments.sample(3)
# Création d'un fichier csv de la table aliments

#Calcul de la disponibilité en kcal par an pour tous les habitants du pays 
aliments['Disponibilté alimentaire en kcal par an'] = aliments['dispo_alim_kcal_p_j'] * aliments['population'] * 365

#Disponibilité des protéines en kg par an pour tous les habitants du pays
aliments['Disponibilté proteines en kg par an'] = (aliments['dispo_prot']/1000 )* aliments['population']*365



(174, 14)
Résultat :  22 colonnes,  4764 lignes


In [1718]:
#Calcul du ratio en énergie et le poids par an pour chaque pays
aliments['Ratio/Energie/Poids'] = (aliments['dispo_alim_kcal_p_j'] * 365) / aliments['Disponibilité alimentaire en quantité (kg/personne/an)']
aliments.replace(np.nan, 0, inplace=True)
aliments


,Code Pays,Pays,Produit,Code Produit,Année,Origine,alim_ani,Autres Utilisations,dispo_alim_kcal_p_j,Disponibilité alimentaire en quantité (kg/personne/an),...,nourriture,pertes,Production,semences,transfo,Variation de stock,population,Disponibilté alimentaire en kcal par an,Disponibilté proteines en kg par an,Ratio/Energie/Poids
0,2,Afghanistan,Abats Comestible,2736,2013,Animale,0.0,0.0,5.0,1.72,...,53.0,0.0,53.0,0.0,0.0,0.0,30552,55757400.0,8586.6396,1061.046512
1,2,Afghanistan,"Agrumes, Autres",2614,2013,Végétale,0.0,0.0,1.0,1.29,...,39.0,2.0,3.0,0.0,0.0,0.0,30552,11151480.0,223.0296,282.945736
2,2,Afghanistan,Aliments pour enfants,2680,2013,Végétale,0.0,0.0,1.0,0.06,...,2.0,0.0,0.0,0.0,0.0,0.0,30552,11151480.0,334.5444,6083.333333
3,2,Afghanistan,Ananas,2618,2013,Végétale,0.0,0.0,0.0,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,30552,0.0,0.0000,0.000000
4,2,Afghanistan,Bananes,2615,2013,Végétale,0.0,0.0,4.0,2.70,...,82.0,0.0,0.0,0.0,0.0,0.0,30552,44605920.0,557.5740,540.740741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4759,256,Luxembourg,Viande de Suides,2733,2013,Animale,0.0,0.0,328.0,43.58,...,23.0,0.0,11.0,0.0,0.0,0.0,530,63451600.0,2331.0725,2747.131712
4760,256,Luxembourg,Viande de Volailles,2734,2013,Animale,0.0,0.0,75.0,21.37,...,11.0,0.0,2.0,0.0,0.0,0.0,530,14508750.0,1479.8925,1281.001404
4761,256,Luxembourg,"Viande, Autre",2735,2013,Animale,0.0,0.0,7.0,2.02,...,1.0,0.0,0.0,0.0,0.0,0.0,530,1354150.0,183.7775,1264.851485
4762,256,Luxembourg,Vin,2655,2013,Végétale,0.0,0.0,100.0,51.41,...,27.0,0.0,10.0,0.0,0.0,0.0,530,19345000.0,1.9345,709.978603


In [1719]:
# question 5 : 5 aliments les plus calorique
df_max_cal = aliments[['Produit', 'Ratio/Energie/Poids']]
df_max_cal = df_max_cal.sort_values('Ratio/Energie/Poids', ascending=False)
df_max_cal.replace([np.inf, -np.inf], np.nan, inplace=True)
df_max_cal = df_max_cal.dropna(subset=['Ratio/Energie/Poids'])
df_max_cal = df_max_cal.drop_duplicates('Produit')
df_max_cal.head(5)

,Produit,Ratio/Energie/Poids
1703,Huil Plantes Oleif Autr,18250.0
1540,Huile d'Olive,18250.0
725,Huile de Tournesol,18250.0
3507,Huiles de Foie de Poisso,18250.0
4182,Huile d'Arachide,18250.0


In [1720]:
# question 5 : 5 aliments les plus riches en protéines
df_max_prot = aliments[['Produit', 'Ratio/Energie/Poids', 'Disponibilté alimentaire en kcal par an', 'Disponibilté proteines en kg par an']]
df_max_prot['densite_prot'] = 4* (df_max_prot['Disponibilté proteines en kg par an'] / df_max_prot['Disponibilté alimentaire en kcal par an']) * df_max_prot['Ratio/Energie/Poids']
df_max_prot = df_max_prot[df_max_prot['densite_prot'].notna()]
df_max_prot = df_max_prot.sort_values('densite_prot', ascending=False)
df_max_prot.replace([np.inf, -np.inf], np.nan, inplace=True)
df_max_prot = df_max_prot.dropna(subset='densite_prot')
df_max_prot = df_max_prot.drop_duplicates('Produit')
df_max_prot.head(5)

/tmp/ipykernel_6637/844591520.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_max_prot['densite_prot'] = 4* (df_max_prot['Disponibilté proteines en kg par an'] / df_max_prot['Disponibilté alimentaire en kcal par an'])


,Produit,Ratio/Energie/Poids,Disponibilté alimentaire en kcal par an,Disponibilté proteines en kg par an,densite_prot
856,Thé,280.769231,7764645.0,2795.2722,0.00144
1874,Crustacés,337.962963,2955040.0,827.4112,0.00112
3383,Cephalopodes,536.764706,35913810.0,9337.5906,0.00104
3246,Perciform,380.208333,2820720.0,648.7656,0.00092
3273,"Viande, Autre",1079.881657,7051800.0,1621.9140,0.00092


In [1721]:
# question 6 : disponibilité mondiale en millier de tonne (masse)
# poids moyen d'un être humain : 62 kg (masse )
# besoin calorique = metabolisme de base * 1.37 ~ 2000 kcal (énergie)
# masse moyenne théorique de population mondiale = 62 * pop_tot = 6997326000 * 62 = 433834212000 kg
df_dispo_mond = aliments[['dispo_int', 'Ratio/Energie/Poids', 'Origine','Disponibilté proteines en kg par an']]
df_dispo_mond = df_dispo_mond[df_dispo_mond.Origine != 'Animale']
df_dispo_mond['dispo_int_kcal'] = (10**6 * df_dispo_mond['dispo_int']) * df_dispo_mond['Ratio/Energie/Poids']
df_dispo_mond.replace([np.inf, -np.inf], 0, inplace=True)
df_dispo_mond = df_dispo_mond.dropna(subset=['dispo_int_kcal'])
# Question 7
pop_tot = 6997326000 
cal_h_an = 2000 * 365
prot_h_an = 62 * 365
dispo_mond = df_dispo_mond['dispo_int_kcal'].sum()
print(dispo_mond/10**16)

0.17638848157977344


In [1722]:
# Question 8 : Potentiel alimentaire des végétaux
# Résultat : impact important


In [1723]:
# Question 9 

df_dispo_mond = aliments[['Produit','Origine','dispo_int','Disponibilté proteines en kg par an par pays','Disponibilté National en kcal par an par population','Ratio/Energie/Poids', 'pertes', 'alim_ani']]
dispo_alim_mond = df_dispo_mond['Disponibilté National en kcal par an par population'].sum()
dispo_prot_mond = df_dispo_mond['Disponibilté proteines en kg par an par pays'].sum()

print(f"dispo alimentaire mondiale totale : {dispo_alim_mond} kcal par an") 
print(f"dispo protéique alimentaire mondiale : {dispo_prot_mond} kg par an")
print(f"Pourcentage de la population concernée : {(dispo_alim_mond / pop_tot)*100} %")
print(f"Pourcentage de la population concernée : {(dispo_prot_mond / pop_tot)*100} %")

KeyError: "['Disponibilté proteines en kg par an par pays', 'Disponibilté National en kcal par an par population'] not in index"

In [ ]:
# Question 10 : sous nutrition
df_malnutrition = pd.read_csv('fao_2013/FAOSTAT_2013_sous_alimentation.csv', names=names, header=0)
print(df_malnutrition.columns)
df_malnutrition = df_malnutrition[df_malnutrition.Pays != 'Chine']
sum_malnutrition = df_malnutrition['Valeur'].sum()
print(f"{100 * (sum_malnutrition * 10**6 / pop_tot)} % de la population mondiale souffre de sous nutrition")

Index(['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code Année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole'],
      dtype='object')
10.571181048303309 % de la population mondiale souffre de sous nutrition


In [ ]:
# Question 11 : Etude sur les céréales
df_cereal = pd.read_csv('fao_2013/FAOSTAT_2013_cereal.csv', names=names, header=0)
cereals = df_cereal[['Élément', 'Code Produit', 'Produit', 'Valeur']]
cereals = cereals.pivot_table('Valeur', index=['Code Produit','Produit'], columns='Élément')
cereals = cereals[['Nourriture','Aliments pour animaux']]
cereals['Propor_anim'] = 100 * (cereals['Aliments pour animaux'] / (cereals['Nourriture'] + cereals['Aliments pour animaux']))
sum_anim = 100 * cereals['Aliments pour animaux'].sum() / (cereals['Nourriture'].sum() + cereals['Aliments pour animaux'].sum())
print(sum_anim)
cereals


50.48833433481695


,Élément,Nourriture,Aliments pour animaux,Propor_anim
Code Produit,Produit,,,
2511,Blé,3126.914286,1563.620000,33.335648
2513,Orge,43.931250,785.491525,94.703395
2514,Maïs,788.315789,4267.792683,84.408646
2515,Seigle,43.261538,90.305263,67.610560
2516,Avoine,26.063291,136.186992,83.936366
2517,Millet,367.292308,32.848000,8.209120
2518,Sorgho,412.825397,200.044776,32.640645
2520,"Céréales, Autres",33.502924,141.697842,80.877410
2805,Riz (Eq Blanchi),2782.920000,557.451220,16.688301


In [ ]:
# Question 12 
df_malnutrition = df_malnutrition[df_malnutrition['Valeur'].notna()]
df_malnutrition = df_malnutrition[['Code Pays', 'Pays', 'Valeur']]
df_malnutrition = pd.merge(aliments, df_malnutrition, on=['Code Pays', 'Pays'])
df_malnutrition.rename(columns={'Valeur_y':'malnutrition'}, inplace=True)
print(df_malnutrition.columns)
df_malnutrition = df_malnutrition[['Produit','Exportations - Quantité']].sort_values('Exportations - Quantité', ascending=False)
df_malnutrition = df_malnutrition.drop_duplicates('Produit')
print(df_malnutrition.head(15))


Index(['Code Pays', 'Pays', 'Produit', 'Code Produit', 'Année', 'Origine',
       'alim_ani', 'Autres Utilisations', 'dispo_alim_kcal_p_j',
       'Disponibilité alimentaire en quantité (kg/personne/an)',
       'dispo_mat_gr', 'dispo_prot', 'dispo_int', 'Exportations - Quantité',
       'Importations - Quantité', 'nourriture', 'pertes', 'Production',
       'semences', 'transfo', 'Variation de stock', 'population',
       'Disponibilté National en kcal par an par population',
       'Disponibilté proteines en kg par an par pays', 'Ratio/Energie/Poids',
       'Valeur'],
      dtype='object')
                 Produit  Exportations - Quantité
3225              Manioc                   8973.0
3250    Riz (Eq Blanchi)                   6951.0
2386                Soja                   5117.0
2408             Bananes                   3268.0
2357                Maïs                   2880.0
601               Ananas                   2301.0
1135       Sucre Eq Brut                   1969.0
